In [1]:
import tensorflow as tf

import numpy as np

import pandas as pd

### Reading the file

In [2]:
my_data = pd.read_csv('POP.csv')

### Converting the file into numpy

In [3]:
my_data_use = my_data['value'].to_numpy()

In [4]:
my_data_use

array([156309.   , 156527.   , 156731.   , 156943.   , 157140.   ,
       157343.   , 157553.   , 157798.   , 158053.   , 158306.   ,
       158451.   , 158757.   , 158973.   , 159170.   , 159349.   ,
       159556.   , 159745.   , 159956.   , 160184.   , 160449.   ,
       160718.   , 160978.   , 161223.   , 161453.   , 161690.   ,
       161912.   , 162124.   , 162350.   , 162564.   , 162790.   ,
       163026.   , 163290.   , 163570.   , 163847.   , 164107.   ,
       164349.   , 164588.   , 164809.   , 165018.   , 165251.   ,
       165463.   , 165695.   , 165931.   , 166192.   , 166473.   ,
       166755.   , 167023.   , 167270.   , 167513.   , 167746.   ,
       167977.   , 168221.   , 168436.   , 168659.   , 168903.   ,
       169191.   , 169488.   , 169780.   , 170063.   , 170315.   ,
       170571.   , 170806.   , 171029.   , 171271.   , 171501.   ,
       171741.   , 171984.   , 172257.   , 172538.   , 172816.   ,
       173070.   , 173298.   , 173533.   , 173746.   , 173945.

### Creating the window and horizon

### Hyper-parameters to be considered 

--> Change the value of 7 in my_target if you are changing the window size ( Window size is the sequence length)

--> Modify the window size (sequence_length to see if there is a difference in the model's accuracy)

In [5]:
my_data = my_data_use
my_target = my_data_use[7:] 
Dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    my_data, my_target, sequence_length=7,sequence_stride= 1, sampling_rate=1, batch_size=len(my_data_use))

len(Dataset)

1

In [6]:
for batch in Dataset:
    inputs, targets = batch
    break
    
len(targets)

809

In [7]:
for i in range(3):
    print(f'window : {inputs[i]} ----> Label {targets[i]}')

window : [156309. 156527. 156731. 156943. 157140. 157343. 157553.] ----> Label 157798.0
window : [156527. 156731. 156943. 157140. 157343. 157553. 157798.] ----> Label 158053.0
window : [156731. 156943. 157140. 157343. 157553. 157798. 158053.] ----> Label 158306.0


In [8]:
for i in range(3):
    print(f'window : {inputs[i-3]} ----> Label {targets[i-3]}')

window : [328742.843 328890.25  329047.319 329213.989 329394.993 329591.333
 329785.872] ----> Label 329982.035
window : [328890.25  329047.319 329213.989 329394.993 329591.333 329785.872
 329982.035] ----> Label 330154.949
window : [329047.319 329213.989 329394.993 329591.333 329785.872 329982.035
 330154.949] ----> Label 330309.946


### Checking if the values are seperated correctly

In [9]:
my_data_use[-1] == targets[-1]

<tf.Tensor: shape=(), dtype=bool, numpy=True>

### Train and split of the data 

In [10]:
def make_train_test_splits(windows = inputs , labels = targets, test_split = 0.2):
    
    split_size = int(len(windows) * (1-test_split))
    
    train_windows = windows[:split_size]
    
    train_labels = labels[:split_size]
    
    test_windows = windows[split_size:]
    
    test_labels = labels[split_size:]
                         
    return train_windows ,test_windows, train_labels , test_labels

In [11]:
train_windows ,test_windows, train_labels , test_labels = make_train_test_splits()

In [12]:
len(train_windows) , len(test_windows) , len(train_labels), len(test_labels)

(647, 162, 647, 162)

## Model 1- Simple Dense layers 

### Model check point is initiated 

In [13]:
import os 
def create_model_checkpoint(model_name , save_path = 'model_experiments'):
    return tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(save_path, model_name),
                                             verbosity = 0, # Not print the output,
                                             save_best_only = True)

### Creation of the model

In [21]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (7,))


x = layers.Dense(128, activation = 'relu')(inputs)

outputs = layers.Dense(1 , activation = 'linear')(x)

model_1 = tf.keras.Model(inputs, outputs , name = 'model_1_dense')

### Check model_summary

In [22]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7)]               0         
                                                                 
 dense (Dense)               (None, 128)               1024      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


### Compile and fit the model

In [23]:
model_1.compile(loss='mae',
              optimizer='adam',
              metrics=['mae','mse'])

model_1_history = model_1.fit(x = train_windows,
                              y = train_labels,
                              epochs = 100,
                              verbose = 1,
                              batch_size = 128,
                             validation_data = (test_windows, test_labels),
                             callbacks =[create_model_checkpoint(
                                         model_name = 'Dense.h5')])

Epoch 1/100
6/6 [==============================] - 1s 89ms/step - loss: 215434.0469 - mae: 215434.0469 - mse: 48336777216.0000 - val_loss: 213814.3281 - val_mae: 213814.3281 - val_mse: 45752786944.0000
Epoch 2/100
6/6 [==============================] - 0s 54ms/step - loss: 122539.5000 - mae: 122539.5000 - mse: 15892895744.0000 - val_loss: 88734.9844 - val_mae: 88734.9844 - val_mse: 7879870464.0000
Epoch 3/100
6/6 [==============================] - 0s 62ms/step - loss: 34715.3828 - mae: 34715.3828 - mse: 1694821376.0000 - val_loss: 27630.8301 - val_mae: 27630.8301 - val_mse: 764250432.0000
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 30430.5840 - mae: 30430.5840 - mse: 997399552.0000 - val_loss: 47392.1406 - val_mae: 47392.1406 - val_mse: 2248128000.0000
Epoch 5/100
6/6 [==============================] - 0s 68ms/step - loss: 24483.8184 - mae: 24483.8184 - mse: 670880000.0000 - val_loss: 1299.9722 - val_mae: 1299.9722 - val_mse: 1698212.5000
Epoch 6/100
6/6 [===

6/6 [==============================] - 0s 7ms/step - loss: 1487.1078 - mae: 1487.1078 - mse: 2443056.2500 - val_loss: 498.2735 - val_mae: 498.2735 - val_mse: 259788.2656
Epoch 41/100
6/6 [==============================] - 0s 7ms/step - loss: 594.2890 - mae: 594.2890 - mse: 440682.8438 - val_loss: 1883.6708 - val_mae: 1883.6708 - val_mse: 3575492.2500
Epoch 42/100
6/6 [==============================] - 0s 7ms/step - loss: 592.0682 - mae: 592.0682 - mse: 578369.8125 - val_loss: 962.9863 - val_mae: 962.9863 - val_mse: 947423.8125
Epoch 43/100
6/6 [==============================] - 0s 7ms/step - loss: 369.9393 - mae: 369.9393 - mse: 192679.3750 - val_loss: 610.2427 - val_mae: 610.2427 - val_mse: 390088.6250
Epoch 44/100
6/6 [==============================] - 0s 7ms/step - loss: 295.4300 - mae: 295.4300 - mse: 119808.3203 - val_loss: 745.4440 - val_mae: 745.4440 - val_mse: 574270.8750
Epoch 45/100
6/6 [==============================] - 0s 74ms/step - loss: 296.8573 - mae: 296.8573 - mse: 11

Epoch 83/100
6/6 [==============================] - 0s 10ms/step - loss: 654.8939 - mae: 654.8939 - mse: 694208.1875 - val_loss: 1046.8872 - val_mae: 1046.8872 - val_mse: 1116634.5000
Epoch 84/100
6/6 [==============================] - 0s 10ms/step - loss: 1011.6508 - mae: 1011.6508 - mse: 1412802.0000 - val_loss: 1773.8141 - val_mae: 1773.8141 - val_mse: 3172740.0000
Epoch 85/100
6/6 [==============================] - 0s 7ms/step - loss: 1272.3655 - mae: 1272.3655 - mse: 1805816.1250 - val_loss: 1075.0021 - val_mae: 1075.0021 - val_mse: 1176491.7500
Epoch 86/100
6/6 [==============================] - 0s 7ms/step - loss: 1361.5850 - mae: 1361.5850 - mse: 2525488.0000 - val_loss: 514.2029 - val_mae: 514.2029 - val_mse: 281465.0312
Epoch 87/100
6/6 [==============================] - 0s 7ms/step - loss: 750.3391 - mae: 750.3391 - mse: 887064.8125 - val_loss: 99.8927 - val_mae: 99.8927 - val_mse: 17975.0293
Epoch 88/100
6/6 [==============================] - 0s 7ms/step - loss: 824.4307 - 

### Evaluation of the model

In [24]:
model_1.evaluate(test_windows,test_labels)

6/6 [==============================] - 0s 2ms/step - loss: 282.7649 - mae: 282.7649 - mse: 92495.8125


[282.7648620605469, 282.7648620605469, 92495.8125]

### Using callback to bring the best model 

In [30]:
model_1 = tf.keras.models.load_model('model_experiments/model_1_dense/')

### Helper function's to check the accuracy of the model 

In [14]:
def make_pred(model, input_data):
    
    forecast = model.predict(input_data)
    
    return tf.squeeze(forecast
                     )




In [15]:
from tensorflow.keras.metrics import mean_absolute_error, mean_squared_error , mean_absolute_percentage_error

def metrics( y_true, y_pred):
    
    #y_true = tf.cast(y_true, dtype = tf.float32)
    #y_pred = tf.cast(y_true, dtype = tf.float32)
    
    MAE = mean_absolute_error(y_true, y_pred)
    
    MSE = mean_squared_error(y_true, y_pred)
    
    RMSE = tf.sqrt(MSE)
    
    MAPE = mean_absolute_percentage_error(y_true, y_pred)
    
    #MASE1 = MASE(y_true, y_pred)
    
    
    return {f'MAE : {MAE.numpy()}', 
            f'MSE : {MSE}',
            f'RMSE : {RMSE} ',
            f'MAPE : {MAPE} '}

In [16]:
model_1_preds = make_pred(model_1 , test_windows)

model1_results = metrics(y_true = test_labels , y_pred = model_1_preds)

model1_results

NameError: name 'model_1' is not defined

## Model 2- LSTM use the default activation function by not entering anything and then try tanh

### Here we need to adjust the size so we added the lambda layer 

In [22]:
from tensorflow.keras import layers

tf.random.set_seed(42)

inputs = layers.Input(shape = (7,1))

#x = tf.expand_dims(inputs, axis=1)(inputs)

x = layers.LSTM(128 , activation = 'relu')(inputs)        #      

#x = layers.LSTM(64 , activation = 'relu')(x)

x = layers.Dense(32 , activation = 'relu')(x)

outputs = layers.Dense(1 , activation = 'linear')(x)

model_2 = tf.keras.Model(inputs, outputs , name = 'model_2_LSTM')

In [23]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 7, 1)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 70,721
Trainable params: 70,721
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile model
model_2.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam())

# Fit model
model_2.fit(train_windows,
            train_labels,
            batch_size=128, 
            epochs=100,
            verbose=1,
            validation_data=(test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name='LSTM.h5')])

Epoch 1/100
6/6 [==============================] - 1s 54ms/step - loss: 14815.4775 - val_loss: 16705.0801
Epoch 2/100
6/6 [==============================] - 0s 30ms/step - loss: 19130.7363 - val_loss: 2150.3479
Epoch 3/100
6/6 [==============================] - 0s 25ms/step - loss: 2091.5984 - val_loss: 3079.8411
Epoch 4/100
6/6 [==============================] - 0s 30ms/step - loss: 931.6116 - val_loss: 2057.6167
Epoch 5/100
6/6 [==============================] - 0s 28ms/step - loss: 671.9607 - val_loss: 1142.5872
Epoch 6/100
6/6 [==============================] - 0s 29ms/step - loss: 489.5757 - val_loss: 617.4373
Epoch 7/100
6/6 [==============================] - 0s 25ms/step - loss: 769.9193 - val_loss: 1298.5264
Epoch 8/100
6/6 [==============================] - 0s 29ms/step - loss: 708.9448 - val_loss: 588.2400
Epoch 9/100
6/6 [==============================] - 0s 25ms/step - loss: 737.4559 - val_loss: 925.2505
Epoch 10/100
6/6 [==============================] - 0s 25ms/step - los

6/6 [==============================] - 0s 23ms/step - loss: 1482.5551 - val_loss: 538.9471
Epoch 80/100
6/6 [==============================] - 0s 25ms/step - loss: 352.8414 - val_loss: 324.3137
Epoch 81/100
6/6 [==============================] - 0s 26ms/step - loss: 638.8606 - val_loss: 299.7564
Epoch 82/100
6/6 [==============================] - 0s 23ms/step - loss: 870.2285 - val_loss: 2666.6208
Epoch 83/100
6/6 [==============================] - 0s 24ms/step - loss: 1661.3057 - val_loss: 1172.6144
Epoch 84/100
6/6 [==============================] - 0s 24ms/step - loss: 790.7521 - val_loss: 1630.1945
Epoch 85/100
6/6 [==============================] - 0s 23ms/step - loss: 851.8128 - val_loss: 271.3522
Epoch 86/100
6/6 [==============================] - 0s 24ms/step - loss: 600.0411 - val_loss: 2255.5952
Epoch 87/100
6/6 [==============================] - 0s 23ms/step - loss: 909.4651 - val_loss: 353.2492
Epoch 88/100
6/6 [==============================] - 0s 23ms/step - loss: 903.464

In [33]:
model_2 = tf.keras.models.load_model('model_experiments/LSTM.h5')

In [34]:
model_2.evaluate(test_windows,test_labels)

6/6 [==============================] - 0s 6ms/step - loss: 127.6418


127.64178466796875

In [35]:
#model_2 = tf.keras.models.load_model('model_experiments/model_2_LSTM/')

model_2_preds = make_pred(model_2 , test_windows)

model2_results = metrics(y_true = test_labels , y_pred = model_2_preds)

model2_results

6/6 [==============================] - 0s 6ms/step


{'MAE : 127.64178466796875',
 'MAPE : 0.04038430377840996 ',
 'MSE : 22723.728515625',
 'RMSE : 150.74391174316406 '}

## Model 3 using the conv1D model - Lambda layers are not accepted so they are removed and 1 is added after 7 (For extra dimension)

In [45]:
tf.random.set_seed(42)

inputs = layers.Input(shape = (7,1)) # 1 is added for extra dimension

x = layers.Lambda(lambda y: tf.expand_dims(y, axis=1)) (inputs)

x = layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu") (x)       #      

#x = layers.LSTM(64 , activation = 'relu')(x)

x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(32 , activation = 'relu')(x)

outputs = layers.Dense(1 , activation = 'linear')(x)

model_3 = tf.keras.Model(inputs, outputs , name = 'model_3_CONV1D')

In [46]:
model_3.summary()

Model: "model_3_CONV1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 7)]               0         
                                                                 
 lambda_3 (Lambda)           (None, 1, 7)              0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 128)            4608      
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_8 (Dense)             (None, 32)                4128      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                    

In [47]:
# Compile model
model_3.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam())

# Fit model
model_3.fit(train_windows,
            train_labels,
            batch_size=128, 
            epochs=100,
            verbose=0,
            validation_data=(test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name='Conv1D.h5')])

INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


...

INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


INFO:tensorflow:Assets written to: model_experiments\model_3_CONV1D\assets


In [44]:
model_3

In [48]:
model_3.evaluate(test_windows,test_labels)

6/6 [==============================] - 0s 2ms/step - loss: 2501.9097


2501.90966796875

In [51]:
model_3 = tf.keras.models.load_model('model_experiments/model_3_CONV1D/')

model_3_preds = make_pred(model_3 , test_windows)

model3_results = metrics(y_true = test_labels , y_pred = model_3_preds)

model3_results

6/6 [==============================] - 0s 1ms/step


{'MAE : 113.64718627929688',
 'MAPE : 0.036565784364938736 ',
 'MSE : 21349.921875',
 'RMSE : 146.11611938476562 '}

## Multivariate data window and horizon creation

In [52]:
# Setup dataset hyperparameters
HORIZON = 1
WINDOW_SIZE = 7

### Just create a copy of the file which is already their in our example my_data_use

#### In the place of price put the output variable of the exam's question

In [ ]:
# Make a copy of the Bitcoin historical data with block reward feature
# Add block_reward column
#bitcoin_prices_block = bitcoin_prices.copy()
#bitcoin_prices_block["block_reward"] = None

bitcoin_prices_windowed = bitcoin_prices_block.copy()     

# Add windowed columns
for i in range(WINDOW_SIZE): # Shift values for each step in WINDOW_SIZE
  bitcoin_prices_windowed[f"Price+{i+1}"] = bitcoin_prices_windowed["Price"].shift(periods=i+1)
bitcoin_prices_windowed.head(10)

### Drop the output variable for the x and add only the output variable for the y 

In [ ]:
# Let's create X & y, remove the NaN's and convert to float32 to prevent TensorFlow errors 
X = bitcoin_prices_windowed.dropna().drop("Price", axis=1).astype(np.float32) 
y = bitcoin_prices_windowed.dropna()["Price"].astype(np.float32)
X.head()


### Split the data 

In [ ]:
# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

## Follow the same process as above to execute the model 